In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Intellectual_Disability"
cohort = "GSE63870"

# Input paths
in_trait_dir = "../../input/GEO/Intellectual_Disability"
in_cohort_dir = "../../input/GEO/Intellectual_Disability/GSE63870"

# Output paths
out_data_file = "../../output/preprocess/Intellectual_Disability/GSE63870.csv"
out_gene_data_file = "../../output/preprocess/Intellectual_Disability/gene_data/GSE63870.csv"
out_clinical_data_file = "../../output/preprocess/Intellectual_Disability/clinical_data/GSE63870.csv"
json_path = "../../output/preprocess/Intellectual_Disability/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Identification of markers of early dementia in adults with Down syndrome"
!Series_summary	"We aimed at identification of variations of genome expression in white blood cells, which could serve as blood markers of early dementia in adults with Down syndrome"
!Series_overall_design	"Whole genome expression analysis was compared between groups of younger and older patients with or without severe cognitive disability"
Sample Characteristics Dictionary:
{0: ['age: Young', 'age: Old'], 1: ['condition: severe cognitive disability and early dementia', 'condition: without severe cognitive disability', 'condition: severe cognitive disability'], 2: ['cell type: white blood cell']}


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
import pandas as pd
import os
import json
from typing import Optional, Callable, Dict, Any

# 1. Gene Expression Data Availability
# Based on the series title and summary, this dataset appears to contain genome expression data in white blood cells
# This is likely gene expression data, not just miRNA or methylation
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability
# The trait we're studying is Intellectual_Disability
# From sample characteristics, we can see that row 1 contains information about cognitive disability
trait_row = 1

# Age information is in row 0
age_row = 0

# No gender information is available in the sample characteristics
gender_row = None

# 2.2 Data Type Conversion Functions
def convert_trait(value):
    """Convert cognitive disability status to binary trait value for Intellectual_Disability."""
    if value is None or pd.isna(value):
        return None
    
    # Ensure value is treated as string
    value = str(value)
    
    # Extract the value after the colon if exists
    if ":" in value:
        value = value.split(":", 1)[1].strip()
    
    # Map values to binary 0/1 for Intellectual_Disability
    if "severe cognitive disability" in value.lower() or "early dementia" in value.lower():
        return 1
    elif "without severe cognitive disability" in value.lower():
        return 0
    else:
        return None

def convert_age(value):
    """Convert age information to binary categories."""
    if value is None or pd.isna(value):
        return None
    
    # Ensure value is treated as string
    value = str(value)
    
    # Extract the value after the colon if exists
    if ":" in value:
        value = value.split(":", 1)[1].strip()
    
    # Convert to binary categories based on provided values
    if value.lower() == "young":
        return 0
    elif value.lower() == "old":
        return 1
    else:
        return None

def convert_gender(value):
    """Convert gender information to binary."""
    # Not used in this dataset as gender information is not available
    return None

# 3. Save Metadata - Initial Filtering
# Determine if trait data is available
is_trait_available = trait_row is not None

# Validate and save cohort information for initial filtering
validate_and_save_cohort_info(
    is_final=False, 
    cohort=cohort, 
    info_path=json_path, 
    is_gene_available=is_gene_available, 
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
# Since trait_row is not None, we proceed with clinical feature extraction
if trait_row is not None:
    # Make sure the output directory exists
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    
    # Get the sample characteristics dictionary
    characteristics_dict = {
        0: ['age: Young', 'age: Old'], 
        1: ['condition: severe cognitive disability and early dementia', 'condition: without severe cognitive disability', 'condition: severe cognitive disability'], 
        2: ['cell type: white blood cell']
    }
    
    # Create sample IDs based on the maximum length of characteristics
    max_samples = max(len(values) for values in characteristics_dict.values())
    sample_ids = [f"GSM{i+1}" for i in range(max_samples)]
    
    # Create a properly structured DataFrame for geo_select_clinical_features
    # Here, rows are characteristics and columns are samples
    data = {}
    for row_idx, values in characteristics_dict.items():
        row_data = {}
        for i, sample_id in enumerate(sample_ids):
            if i < len(values):
                row_data[sample_id] = values[i]
            else:
                row_data[sample_id] = None
        data[row_idx] = row_data
    
    clinical_data = pd.DataFrame(data).T
    
    # Extract clinical features
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the extracted clinical features
    preview_data = preview_df(selected_clinical_df)
    print("Preview of extracted clinical features:")
    print(preview_data)
    
    # Save the extracted clinical features to CSV
    selected_clinical_df.to_csv(out_clinical_data_file)
    print(f"Saved clinical data to {out_clinical_data_file}")


Preview of extracted clinical features:
{'GSM1': [1.0, 0.0], 'GSM2': [1.0, 1.0], 'GSM3': [1.0, nan]}
Saved clinical data to ../../output/preprocess/Intellectual_Disability/clinical_data/GSE63870.csv


### Step 3: Gene Data Extraction

In [4]:
# 1. Get the file paths for the SOFT file and matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Extract gene expression data from the matrix file
try:
    print("Extracting gene data from matrix file:")
    gene_data = get_genetic_data(matrix_file)
    if gene_data.empty:
        print("Extracted gene expression data is empty")
        is_gene_available = False
    else:
        print(f"Successfully extracted gene data with {len(gene_data.index)} rows")
        print("First 20 gene IDs:")
        print(gene_data.index[:20])
        is_gene_available = True
except Exception as e:
    print(f"Error extracting gene data: {e}")
    print("This dataset appears to have an empty or malformed gene expression matrix")
    is_gene_available = False

print(f"\nGene expression data available: {is_gene_available}")


Extracting gene data from matrix file:


Successfully extracted gene data with 50739 rows
First 20 gene IDs:
Index(['(+)E1A_r60_1', '(+)E1A_r60_3', '(+)E1A_r60_a104', '(+)E1A_r60_a107',
       '(+)E1A_r60_a135', '(+)E1A_r60_a20', '(+)E1A_r60_a22', '(+)E1A_r60_a97',
       '(+)E1A_r60_n11', '(+)E1A_r60_n9', '3xSLv1', 'A_19_P00315452',
       'A_19_P00315459', 'A_19_P00315482', 'A_19_P00315492', 'A_19_P00315493',
       'A_19_P00315502', 'A_19_P00315506', 'A_19_P00315518', 'A_19_P00315519'],
      dtype='object', name='ID')

Gene expression data available: True


### Step 4: Gene Identifier Review

In [5]:
# Analyzing the gene identifiers in the gene expression data
# Looking at the first 20 gene IDs, I can see:
# 1. Control probes like "(+)E1A_r60_1", "(+)E1A_r60_3", etc.
# 2. Array/probe IDs like "A_19_P00315452", "A_19_P00315459", etc.
# 3. Other identifiers like "3xSLv1"

# These are not standard human gene symbols (like BRCA1, TP53, etc.)
# They appear to be array-specific probe IDs from a microarray platform
# These need to be mapped to standard gene symbols

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Extract gene annotation data from the SOFT file
print("Extracting gene annotation data from SOFT file...")
try:
    # Use the library function to extract gene annotation
    gene_annotation = get_gene_annotation(soft_file)
    print(f"Successfully extracted gene annotation data with {len(gene_annotation.index)} rows")
    
    # Preview the annotation DataFrame
    print("\nGene annotation preview (first few rows):")
    print(preview_df(gene_annotation))
    
    # Show column names to help identify which columns we need for mapping
    print("\nColumn names in gene annotation data:")
    print(gene_annotation.columns.tolist())
    
    # Check for relevant mapping columns
    if 'GB_ACC' in gene_annotation.columns:
        print("\nThe dataset contains GenBank accessions (GB_ACC) that could be used for gene mapping.")
        # Count non-null values in GB_ACC column
        non_null_count = gene_annotation['GB_ACC'].count()
        print(f"Number of rows with GenBank accessions: {non_null_count} out of {len(gene_annotation)}")
    
    if 'SPOT_ID' in gene_annotation.columns:
        print("\nThe dataset contains genomic regions (SPOT_ID) that could be used for location-based gene mapping.")
        print("Example SPOT_ID format:", gene_annotation['SPOT_ID'].iloc[0])
    
except Exception as e:
    print(f"Error processing gene annotation data: {e}")
    is_gene_available = False


Extracting gene annotation data from SOFT file...


Successfully extracted gene annotation data with 2486259 rows

Gene annotation preview (first few rows):
{'ID': ['GE_BrightCorner', 'DarkCorner', 'A_23_P117082', 'A_33_P3246448', 'A_33_P3318220'], 'SPOT_ID': ['CONTROL', 'CONTROL', 'A_23_P117082', 'A_33_P3246448', 'A_33_P3318220'], 'CONTROL_TYPE': ['pos', 'pos', 'FALSE', 'FALSE', 'FALSE'], 'REFSEQ': [nan, nan, 'NM_015987', 'NM_080671', 'NM_178466'], 'GB_ACC': [nan, nan, 'NM_015987', 'NM_080671', 'NM_178466'], 'LOCUSLINK_ID': [nan, nan, 50865.0, 23704.0, 128861.0], 'GENE_SYMBOL': [nan, nan, 'HEBP1', 'KCNE4', 'BPIFA3'], 'GENE_NAME': [nan, nan, 'heme binding protein 1', 'potassium voltage-gated channel, Isk-related family, member 4', 'BPI fold containing family A, member 3'], 'UNIGENE_ID': [nan, nan, 'Hs.642618', 'Hs.348522', 'Hs.360989'], 'ENSEMBL_ID': [nan, nan, 'ENST00000014930', 'ENST00000281830', 'ENST00000375454'], 'ACCESSION_STRING': [nan, nan, 'ref|NM_015987|ens|ENST00000014930|gb|AF117615|gb|BC016277', 'ref|NM_080671|ens|ENST00000

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Determine which columns to use for gene mapping
# Based on the gene expression data and gene annotation preview, we can see:
# - ID column in the annotation contains probe IDs that match the gene expression data index
# - GENE_SYMBOL column contains the human gene symbols we want to map to

probe_id_col = 'ID'
gene_symbol_col = 'GENE_SYMBOL'

print(f"Using {probe_id_col} as probe identifier and {gene_symbol_col} as gene symbol")

# 2. Get a gene mapping dataframe by extracting the relevant columns
print("Creating gene mapping dataframe...")
gene_mapping = get_gene_mapping(gene_annotation, prob_col=probe_id_col, gene_col=gene_symbol_col)
print(f"Created gene mapping with {len(gene_mapping)} entries")
print("Preview of gene mapping:")
print(preview_df(gene_mapping))

# 3. Apply gene mapping to convert probe-level measurements to gene expression data
print("Converting probe-level measurements to gene expression data...")
gene_data = apply_gene_mapping(gene_data, gene_mapping)
print(f"Created gene expression data with {len(gene_data)} genes")

# Preview the gene expression data
print("Preview of gene expression data:")
print(gene_data.head(3))

# Save the gene expression data to a CSV file
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"Saved gene expression data to {out_gene_data_file}")


Using ID as probe identifier and GENE_SYMBOL as gene symbol
Creating gene mapping dataframe...
Created gene mapping with 46204 entries
Preview of gene mapping:
{'ID': ['A_23_P117082', 'A_33_P3246448', 'A_33_P3318220', 'A_33_P3236322', 'A_33_P3319925'], 'Gene': ['HEBP1', 'KCNE4', 'BPIFA3', 'LOC100129869', 'IRG1']}
Converting probe-level measurements to gene expression data...


Created gene expression data with 20353 genes
Preview of gene expression data:
          GSM1558696  GSM1558697  GSM1558698  GSM1558699  GSM1558700  \
Gene                                                                   
A1BG       -1.882027   -1.537594   -1.531755   -0.498507    0.175755   
A1BG-AS1   -0.319541   -0.085776   -0.640145   -0.149423    0.366026   
A1CF       -0.600164   -0.309273   -0.249359    0.435776    0.290430   

          GSM1558701  GSM1558702  GSM1558703  GSM1558704  GSM1558705  ...  \
Gene                                                                  ...   
A1BG        1.084357    0.145071    0.832431    1.468052    2.828776  ...   
A1BG-AS1    1.032678    0.328191   -0.327742   -0.106659    0.174874  ...   
A1CF        0.664791    0.622737   -0.218599    0.195794    0.248855  ...   

          GSM1558734  GSM1558735  GSM1558736  GSM1558737  GSM1558738  \
Gene                                                                   
A1BG        0.850051   -0.9181

Saved gene expression data to ../../output/preprocess/Intellectual_Disability/gene_data/GSE63870.csv


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the gene expression data
print("Normalizing gene symbols in the expression data...")
try:
    # If previous steps have already loaded gene_data
    normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
    
    # Create directory if it doesn't exist
    os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
    
    # Save normalized gene data
    normalized_gene_data.to_csv(out_gene_data_file)
    print(f"Normalized gene data saved to {out_gene_data_file}")
    print(f"Normalized gene data shape: {normalized_gene_data.shape}")
except Exception as e:
    print(f"Error normalizing gene data: {e}")
    is_gene_available = False

# 2. Load the original clinical data and recreate it properly
print("\nExtracting clinical data from source...")
try:
    # Re-extract background information and sample characteristics from matrix file
    background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
    clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
    background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)
    
    # Process the clinical data to extract trait and age information
    print("\nProcessing clinical data...")
    # Extract the sample IDs which should match the gene expression data
    sample_ids = clinical_data['!Sample_geo_accession'].tolist()
    
    # Get feature data from specified rows
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Display information about clinical data
    print(f"Clinical data shape: {selected_clinical_df.shape}")
    print(f"Clinical data index: {selected_clinical_df.index.tolist()}")
    print(f"Clinical data columns: {selected_clinical_df.columns.tolist()}")
    
    # Get gene data sample IDs
    gene_sample_ids = normalized_gene_data.columns.tolist()
    print(f"Gene data has {len(gene_sample_ids)} samples")
    print(f"First few gene data sample IDs: {gene_sample_ids[:5]}")
    
    # Link clinical and genetic data using the geo helper function
    print("\nLinking clinical and genetic data...")
    linked_data = geo_link_clinical_genetic_data(selected_clinical_df, normalized_gene_data)
    print(f"Linked data shape: {linked_data.shape}")
    
    # Check if the linked data has enough samples
    if linked_data.shape[0] < 3:
        raise ValueError("Not enough samples after linking clinical and genetic data")
    
    # 3. Handle missing values systematically
    print("\nHandling missing values...")
    linked_data = handle_missing_values(linked_data, trait)
    print(f"After handling missing values, data shape: {linked_data.shape}")
    
    # 4. Determine whether features are biased
    print("\nChecking for bias in features...")
    is_biased, linked_data = judge_and_remove_biased_features(linked_data, trait)
    
    # 5. Final validation and save metadata
    print("\nPerforming final validation...")
    is_usable = validate_and_save_cohort_info(
        is_final=True,
        cohort=cohort,
        info_path=json_path,
        is_gene_available=is_gene_available,
        is_trait_available=is_trait_available,
        is_biased=is_biased,
        df=linked_data,
        note="Down syndrome study: severe cognitive disability vs without severe cognitive disability."
    )
    
    # 6. Save the linked data if usable
    if is_usable:
        # Create directory if it doesn't exist
        os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
        
        # Save linked data
        linked_data.to_csv(out_data_file)
        print(f"Linked data saved to {out_data_file}")
    else:
        print(f"Dataset not usable for {trait} association studies. Data not saved.")

except Exception as e:
    print(f"Error in data linking or processing: {e}")
    print("Detailed error information:")
    import traceback
    traceback.print_exc()
    
    # Create a minimal dataframe for validation purposes
    linked_data = pd.DataFrame({trait: [0, 1]})
    
    # Perform final validation with appropriate flags
    is_usable = validate_and_save_cohort_info(
        is_final=True,
        cohort=cohort,
        info_path=json_path,
        is_gene_available=is_gene_available, 
        is_trait_available=is_trait_available,
        is_biased=True,  # Not relevant since data isn't usable
        df=linked_data,
        note="Failed to link gene and clinical data: " + str(e)
    )
    print(f"Dataset usability: {is_usable}")

Normalizing gene symbols in the expression data...


Normalized gene data saved to ../../output/preprocess/Intellectual_Disability/gene_data/GSE63870.csv
Normalized gene data shape: (19847, 48)

Extracting clinical data from source...

Processing clinical data...
Clinical data shape: (2, 48)
Clinical data index: ['Intellectual_Disability', 'Age']
Clinical data columns: ['GSM1558696', 'GSM1558697', 'GSM1558698', 'GSM1558699', 'GSM1558700', 'GSM1558701', 'GSM1558702', 'GSM1558703', 'GSM1558704', 'GSM1558705', 'GSM1558706', 'GSM1558707', 'GSM1558708', 'GSM1558709', 'GSM1558710', 'GSM1558711', 'GSM1558712', 'GSM1558713', 'GSM1558714', 'GSM1558715', 'GSM1558716', 'GSM1558717', 'GSM1558718', 'GSM1558719', 'GSM1558720', 'GSM1558721', 'GSM1558722', 'GSM1558723', 'GSM1558724', 'GSM1558725', 'GSM1558726', 'GSM1558727', 'GSM1558728', 'GSM1558729', 'GSM1558730', 'GSM1558731', 'GSM1558732', 'GSM1558733', 'GSM1558734', 'GSM1558735', 'GSM1558736', 'GSM1558737', 'GSM1558738', 'GSM1558739', 'GSM1558740', 'GSM1558741', 'GSM1558742', 'GSM1558743']
Gene dat

After handling missing values, data shape: (48, 19849)

Checking for bias in features...
Quartiles for 'Intellectual_Disability':
  25%: 1.0
  50% (Median): 1.0
  75%: 1.0
Min: 1.0
Max: 1.0
The distribution of the feature 'Intellectual_Disability' in this dataset is severely biased.

Quartiles for 'Age':
  25%: 0.0
  50% (Median): 0.0
  75%: 1.0
Min: 0.0
Max: 1.0
The distribution of the feature 'Age' in this dataset is fine.


Performing final validation...
Dataset not usable for Intellectual_Disability association studies. Data not saved.
